This script solves the "beam problem" from the MMA paper of Krister Svanberg. 


Minimize:
   $0.0624*(x(1) + x(2) + x(3) + x(4) + x(5))$

Subject to:
   $61/(x(1)^3) + 37/(x(2)^3) + 19/(x(3)^3) +  7/(x(4)^3) +  1/(x(5)^3) <= 1,
    1 <= x(j) <= 10, for j=1,..,5.$

In [2]:
# Loading modules
from __future__ import division
from optymus.methods.topological import mma, check_kkt
from typing import Tuple
import numpy as np


def main() -> None:
    # Set numpy print options
    np.set_printoptions(precision=4, formatter={'float': '{: 0.4f}'.format})
    
    # Beam initial settings
    m, n = 1, 5
    eeen = np.ones((n, 1))
    eeem = np.ones((m, 1))
    zeron = np.zeros((n, 1))
    zerom = np.zeros((m, 1))
    xval = 5 * eeen
    xold1 = xval.copy()
    xold2 = xval.copy()
    xmin = eeen.copy()
    xmax = 10 * eeen
    low = xmin.copy()
    upp = xmax.copy()
    move = 1.0
    c = 1000 * eeem
    d = eeem.copy()
    a0 = 1
    a = zerom.copy()
    innerit = 0
    outeriter = 0
    maxoutit = 11
    kkttol = 0

    # kargs = {
    #     "n_variables": 3,
    #     ....
    # }

    # opt = mma(**kargs)

    # opt = Optimizer(
    #     method = 'mma',
    #     f_obj = f,
    #     **kargs
    # )

    
    # Calculate function values and gradients of the objective and constraints functions
    if outeriter == 0:
        f0val, df0dx, fval, dfdx = beam2(xval)
        outvector1 = np.array([outeriter, innerit, f0val, fval])
        outvector2 = xval.flatten()
    
    # The iterations start
    kktnorm = kkttol + 10
    outit = 0

    while kktnorm > kkttol and outit < maxoutit:
        outit += 1
        outeriter += 1
        
        # The MMA subproblem is solved at the point xval:
        xmma, ymma, zmma, lam, xsi, eta, mu, zet, s, low, upp = mma(
            m, n, outeriter, xval, xmin, xmax, xold1, xold2, f0val, df0dx, fval, dfdx, low, upp, a0, a, c, d, move)
        
        # Some vectors are updated:
        xold2 = xold1.copy()
        xold1 = xval.copy()
        xval = xmma.copy()
        
        # Re-calculate function values and gradients of the objective and constraints functions
        f0val, df0dx, fval, dfdx = beam2(xval)
        
        # The residual vector of the KKT conditions is calculated
        residu, kktnorm, residumax = check_kkt(
            m, n, xmma, ymma, zmma, lam, xsi, eta, mu, zet, s, xmin, xmax, df0dx, fval, dfdx, a0, a, c, d)
        
        outvector1 = np.array([outeriter, innerit, f0val, fval])
        outvector2 = xval.flatten()

        return {
            "xval": xval,
            "f0val": f0val,
            "fval": fval,
            "niter": outeriter,
            }


def beam2(xval: np.ndarray) -> Tuple[float, np.ndarray, float, np.ndarray]:
    nx = 5
    eeen = np.ones((nx, 1))
    c1 = 0.0624
    c2 = 1
    aaa = np.array([[61.0, 37.0, 19.0, 7.0, 1.0]]).T
    xval2 = xval * xval
    xval3 = xval2 * xval
    xval4 = xval2 * xval2
    xinv3 = eeen / xval3
    xinv4 = eeen / xval4
    f0val = c1 * np.dot(eeen.T, xval).item()
    df0dx = c1 * eeen
    fval = (np.dot(aaa.T, xinv3) - c2).item()
    dfdx = -3 * (aaa * xinv4).T
    return f0val, df0dx, fval, dfdx


if __name__ == "__main__":
    res = main()
    print(res)

{'xval': array([[ 5.6966],
       [ 5.1410],
       [ 4.3965],
       [ 3.3533],
       [ 1.8596]]), 'f0val': 1.2758901894116546, 'fval': 0.16701618803366713, 'niter': 1}
